In [1]:
import pandas as pd

In [2]:
# Save the cvs as a dataframe and remove the rows with null variety
df = pd.read_csv('stopword_filtered.csv')
df = df.dropna(subset=['variety'])
df.isnull().any()

Unnamed: 0     False
_c0            False
country         True
description    False
variety        False
Words          False
filtered       False
dtype: bool

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
count_vect = CountVectorizer()
tfidf = TfidfTransformer()
cv = CountVectorizer().fit_transform(df.filtered)
bagofwords = tfidf.fit_transform(cv)

In [4]:
data = df.values
data[0][4]
y = data[:,4]
y

array(['White Blend', 'White Blend', 'White Blend', ..., 'Tempranillo',
       'Tempranillo', 'Tempranillo'], dtype=object)

In [5]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [6]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

/anaconda3/envs/PythonData/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [14]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(bagofwords[:80000], one_hot_y[:80000])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [15]:
predictions = rf.predict(bagofwords) 
len(predictions)

103320

In [16]:
import numpy as np
def decode(datum):
    return np.argmax(datum)

##index = np.argmax(predictions, axis=0)
decoded_var = []
for i in range(predictions.shape[0]):
    pred_var = predictions[i]
    decoded_var.append(decode(predictions[i]))

In [17]:
from sklearn.metrics import accuracy_score
y_pred = decoded_var
y_true = encoded_y
accuracy_score(y_true, y_pred)

0.770876887340302

In [ ]:
X = bagofwords[:10000]
y = encoded_y[:10000]
y.shape

In [ ]:
# Trying Linear SVM
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(X, y)

In [ ]:
x_min = X[:, 0].min()
x_max = X[:, 0].max()
y_min = X[:, 1].min()
y_max = X[:, 1].max()

XX, YY = np.mgrid[x_min:x_max, y_min:y_max]
Z = model.decision_function(np.c_[XX.ravel(), YY.ravel()])

# Put the result into a color plot
Z = Z.reshape(XX.shape)
# plt.pcolormesh(XX, YY, Z > 0, cmap=plt.cm.Paired)
plt.contour(XX, YY, Z, colors=['k', 'k', 'k'],
           linestyles=['--', '-', '--'], levels=[-.5, 0, .5])
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr', edgecolor='k', s=100)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [25]:
model = SVC (kernel='linear')
model.fit(X_train, y_train)
predictions = model.predict(X_test)

(10000, 47)

In [ ]:
# Plot the decision boundaries
x_min = X[:, 0].min()
x_max = X[:, 0].max()
y_min = X[:, 1].min()
y_max = X[:, 1].max()

XX, YY = np.mgrid[x_min:x_max, y_min:y_max]
Z = model.decision_function(np.c_[XX.ravel(), YY.ravel()])

# Put the result into a color plot
Z = Z.reshape(XX.shape)
# plt.pcolormesh(XX, YY, Z > 0, cmap=plt.cm.Paired)
plt.contour(XX, YY, Z, colors=['k', 'k', 'k'],
           linestyles=['--', '-', '--'], levels=[-.5, 0, .5])
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr', edgecolor='k', s=100)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions, target_names=["blue","red"]))